In [ ]:


import itertools
import os

T_SUPPORT = 3
T_CONFIDENCE = 65

callgraph = {}
# key: class, value: {method}

methodSupport = {} 
# key: method name, value: its support

list_of_pairs = []
list_of_pairsupport = []

bugConfidence = {} 
# key: potential bug method, value: [itspair, support, confidence] note: only contain pairs that meet the threshold criteria



In [ ]:
# construct a callgraph map from txt file
# Class: set(Method)

with open("commons-math3-3.6.1.txt") as file:
    for line in file:

        if (line.startswith("C:")):
            cla = line.split(' ',)[0]
            callgraph[cla[2:]] = set()

        if (line.startswith("M:")):
            (cla, metho) = line.split()
            if metho.startswith("(M)"):
                cla = cla[2:].split(':')[0]
                metho = metho[3:].split('(')[0]
                callgraph[cla] = callgraph.get(cla, set()) | {metho}
                methodSupport[metho] = 0

#print(callgraph)

FileNotFoundError: ignored

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#enumerate the callgraph to get the support of every method, forming a map

for i, (j, k) in enumerate(methodSupport.items()):
    for l, m in enumerate(callgraph.values()):
        if j in m:
            methodSupport[j] += 1

#print(methodSupport)

# support filter >= 3

methodSupportfiltered = {k: v for k, v in methodSupport.items() if v >= T_SUPPORT}

#print(methodSupportfiltered)


In [ ]:
# print(methodSupportfiltered)

# initialize pairlist
msl = methodSupportfiltered.keys()

# find all possible pairs
for pair in itertools.combinations(msl, r=2):
    list_of_pairs.append([*pair])

# create a new list containing the support of each pairs

list_of_pairsupport = [0] * len(list_of_pairs)

for i, j in enumerate(list_of_pairs):
    for k in callgraph.values():
        if j[0] in k and j[1] in k:
            list_of_pairsupport[i] += 1


#print(list_of_pairs)
#print(list_of_pairsupport)
#print(callgraph)


In [ ]:
# since we set thershold to support 3. bugs can only be those methods in list_of_pairs

# since the outer loop is filtered and pruned and always set to be much smaller than the whole callgraph, time complexity is not bad

#list_of_pbugs = []

for i, p in enumerate(list_of_pairs):
    pairsup = list_of_pairsupport[i]
    if pairsup >= 3: # set threshold
        for cl, m in callgraph.items():
            if p[0] in m and p[1] not in m:
                con = 100 * pairsup / methodSupport[p[0]]
                if con >= T_CONFIDENCE:
                    print('bug: ' + p[0] + ' in ' + cl + ', pair: (' + p[0] + ', ' +p[1] + '), support: ' + str(pairsup) + ', confidence: ' + str(con) + '%')
            if p[1] in m and p[0] not in m:
                con = pairsup / methodSupport[p[1]]
                if con >= T_CONFIDENCE:
                    print('bug: ' + p[1] + ' in ' + cl + ', pair: (' + p[0] + ', ' +p[1] + '), support: ' + str(pairsup) + ', confidence: ' + str(con) + '%')
